In [29]:
# To add a new cell, type '# %%'
# To add a new markdown cell, type '# %% [markdown]'
# %%
import pandas as pd
import sqlite3
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from sklearn import preprocessing
import numpy as np
from matplotlib import pyplot as ptl
import seaborn as sns
import sys, os
sys.path.append("../module/")
from lipinski import * 

In [30]:
pd.options.display.max_columns = 30

In [31]:
con = sqlite3.connect("../data/chembl_29.db")

In [32]:
molregno_chembl_id = pd.read_sql_query("SELECT compound_properties.*, molecule_dictionary.chembl_id, compound_structures.canonical_smiles FROM compound_properties INNER JOIN compound_structures ON compound_structures.molregno=compound_properties.molregno INNER JOIN molecule_dictionary ON molecule_dictionary.molregno=compound_properties.molregno;", con)

In [33]:
molregno_chembl_id = molregno_chembl_id.drop(['ro3_pass', 'hba', 'hbd', 'num_ro5_violations', 'full_molformula'], axis=1)

In [34]:
molregno_chembl_id = pd.merge(molregno_chembl_id.chembl_id, molregno_chembl_id)

In [35]:
molregno_chembl_id = molregno_chembl_id.drop(['molregno'], axis=1)

In [45]:
molregno_chembl_id = molregno_chembl_id.sort_values(by='psa').iloc[len(molregno_chembl_id) - 300:len(molregno_chembl_id), :]

In [46]:
index_duplicates = molregno_chembl_id[molregno_chembl_id.canonical_smiles.duplicated()].sort_values(by=['canonical_smiles']).index

In [47]:
len(molregno_chembl_id)

300

In [48]:
molregno_chembl_id = molregno_chembl_id.drop(labels=index_duplicates, axis=0)
molregno_chembl_id

,chembl_id,mw_freebase,alogp,psa,rtb,cx_most_apka,cx_most_bpka,cx_logp,cx_logd,molecular_species,full_mwt,aromatic_rings,heavy_atoms,qed_weighted,mw_monoisotopic,hba_lipinski,hbd_lipinski,num_lipinski_ro5_violations,canonical_smiles
2080164,CHEMBL4646183,1471.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,1471.76,NaN,NaN,NaN,1470.7470,NaN,NaN,NaN,CCN(CC)c1ccc2c(-c3ccccc3C(=O)N(C)CCCC(=O)N[C@H...
2080206,CHEMBL4646225,410.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,410.24,NaN,NaN,NaN,410.0414,NaN,NaN,NaN,COc1ccc2c(c1C(=O)OCOC(C)=O)OB(O)[C@@H](Sc1nncs...
2080273,CHEMBL4646292,390.73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,390.73,NaN,NaN,NaN,390.9878,NaN,NaN,NaN,CCOC(=O)c1cc([Se]c2ccccc2)c2cc(Cl)ccc2n1
2080285,CHEMBL4646304,1064.42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,1064.42,NaN,NaN,NaN,1063.7369,NaN,NaN,NaN,CCCCCCCC[C@H](NC(=O)C(C)(C)NC(=O)C(C)(C)NC(=O)...
2080332,CHEMBL4646351,1116.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,1116.54,NaN,NaN,NaN,1115.8046,NaN,NaN,NaN,CCCCCCCC[C@H](NC(=O)C(C)(C)NC(=O)C(C)(C)NC(=O)...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2084598,CHEMBL4651129,1021.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,1021.31,NaN,NaN,NaN,1020.5749,NaN,NaN,NaN,CC[C@H]1CCC[C@H](O[C@H]2CC[C@H](N(C)CCC[P+](c3...
2084601,CHEMBL4651132,1109.47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,1109.47,NaN,NaN,NaN,1108.5377,NaN,NaN,NaN,CC1(C)C(/C=C/C=C/C=C2\N(CCCCCC(=O)NCCCCCCNc3c(...
2084607,CHEMBL4651138,3284.93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,3284.93,NaN,NaN,NaN,3282.8190,NaN,NaN,NaN,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccc(O)cc1)NC(=O...
2084719,CHEMBL4651250,556.38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,556.38,NaN,NaN,NaN,556.2055,NaN,NaN,NaN,COc1ccc([P+](CCC(=O)Nc2ccc3c(c2)B(O)OC3)(c2ccc...


In [49]:
connn = sqlite3.connect('dados_teste.db')
molregno_chembl_id.to_sql('dados_teste', connn, if_exists='replace', index = False)

In [ ]:
ids_com_nan = molregno_chembl_id.loc[molregno_chembl_id.isnull().any(axis=1)][['chembl_id', 'canonical_smiles']]

In [ ]:
ids_com_nan

In [ ]:
atualiza_data_frame_com_lipinski(molregno_chembl_id, ids_com_nan, 1)

In [ ]:
#or chemb, smiles in zip(ids_com_nan.chembl_id, ids_com_nan.canonical_smiles):
 #   propriedades = verifica_lipinsky(smiles)
 ##   for key in propriedades.keys():
  #      molregno_chembl_id.loc[molregno_chembl_id.chembl_id == chemb, key] = propriedades[key]

In [ ]:
molregno_chembl_id

In [ ]:
molregno_chembl_id.loc[(molregno_chembl_id.isnull().any(axis=1))]

In [ ]:
molregno_chembl_id

In [ ]:
molregno_chembl_id.query("molregno in @ids_com_nan")

In [ ]:
molregno_chembl_id.query("num_ro5_violations != num_lipinski_ro5_violations")

In [ ]:
cruzain_id = pd.read_csv('data/cruzain_id_chemb.csv')

In [ ]:
cruzain_id

In [ ]:
cruzain_id = cruzain_id["Molecule ChEMBL ID"]
cruzain_id

In [ ]:
molregno_chembl_id = molregno_chembl_id.query("chembl_id in @cruzain_id")

In [ ]:
molregno_chembl_id.head()

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
molregno_chembl_id.num_lipinski_ro5_violations.unique()

In [ ]:
molregno_chembl_id.num_ro5_violations.unique()

In [ ]:
molregno_chembl_id[molregno_chembl_id.num_lipinski_ro5_violations == molregno_chembl_id.num_ro5_violations]

In [ ]:
molregno_chembl_id['ro3_pass'] = molregno_chembl_id['ro3_pass'].replace({"Y": 'True', "N": 'False'})

In [ ]:
molregno_chembl_id.query("num_lipinski_ro5_violations == 0")

In [ ]:
molregno_chembl_id.query("num_lipinski_ro5_violations == 0 & ro3_pass == 'True'")

In [ ]:
prep_clust = pd.get_dummies(molregno_chembl_id.query("num_lipinski_ro5_violations == 0"), columns=["molecular_species"])
tab_usada = prep_clust
prep_clust = prep_clust.drop(['chembl_id','molregno', "full_molformula", "canonical_smiles"], axis=1)


In [ ]:
prep_clust.head()

In [ ]:
prep_clust = prep_clust.replace({np.nan: "7"})
prep_clust.head()

In [ ]:
x = preprocessing.normalize(prep_clust, norm='l1')
cluster = KMeans(n_clusters=3, random_state=5)
cluster.fit(x).labels_

In [ ]:
plt.figure(figsize=(10,7))
plt.scatter(x=prep_clust[['mw_freebase']], y=prep_clust.psa, c=cluster.fit(x).labels_, s=50)
plt.xlabel('mw_freebase', fontsize=18)
plt.ylabel('psa', fontsize=18)
plt.show()

In [ ]:
tab_usada['grupos_cluster_knn'] = cluster.fit(x).labels_
tab_usada.drop(["num_ro5_violations", "ro3_pass", "num_lipinski_ro5_violations"], axis=1).sort_values(by='grupos_cluster_knn')

In [ ]:
sns.pairplot(tab_usada[['mw_freebase', 'psa', 'grupos_cluster_knn']], hue='grupos_cluster_knn')
